## 1D QHO

> We want to find the minimum Energy

### Steps

- Program the wavefunction $\Psi(x)=e^{xb}\Gamma(x)$ with $\Gamma(x)=\prod_{j=1}^M(1+e^{c_j+xW_j})$

In [1]:
using Random

In [69]:
N_hidden = 5
visible_bias = 1
W = rand(N_hidden)
# W = repeat([0.5], N_hidden)
hidden_bias = repeat([1], N_hidden);

In [70]:
function Γ(x)
    gamma=1
    for i in 1:N_hidden
        gamma = gamma.*(1 .+ exp.(hidden_bias[i].+x.*W[i]))
    end
    return gamma
end

Γ (generic function with 1 method)

In [71]:
function Ψ(x)
    return exp.(x.*visible_bias).*Γ(x)
end

Ψ (generic function with 1 method)

#### We define the simpson method to integrate

In [72]:
function simps(f::Function, a::Number, b::Number, n::Number)
    n % 2 == 0 || error("`n` must be even")
    h = (b-a)/n
    s = f(a) + f(b)
    s += 4*sum(f(a .+ collect(1:2:n) .* h))
    s += 2*sum(f(a .+ collect(2:2:n-1) .* h))
    return h/3 * s
end

simps (generic function with 1 method)

Test simpson (if working, output really close to 1)

In [73]:
function gauss(x)
    a = 1/(2*3.14159)^(0.5)
    return a.*exp.(-0.5.*x.^2)
end

integral = simps(gauss, -10,10,5000)
println(integral)

1.0000004223322003


In [74]:
normalization = simps(x->(Ψ(x)).^2,-1,1,1000)
normalization


5.3543541217672005e7

> Define auxiliary functions σ and σ_i

In [75]:
function σ(x)
    return 1 ./ (1 .+ exp.(-(x)))
end

function σ_i(x,i::Int)
    return σ(hidden_bias[i].+x.*W[i])
end;

## Kinetic term

$$\frac{\partial ^2}{\partial^2 x} \Psi (x) = \sum_{\{\vec{h}\}}e^{xb+\vec{c}'\vec{h}+x\vec{W}\vec{h}}(b+\vec{W}\vec{h})^2$$

Then we have 3 terms:
- $\eta_1\Psi=b^2\sum_{\{\vec{h}\}}e^{xb+\vec{c}'\vec{h}+x\vec{W}\vec{h}} = b^2\Psi(x)$
- $\eta_2\Psi=2b\vec{W}\sum_{\{\vec{h}\}}e^{xb+\vec{c}'\vec{h}+x\vec{W}\vec{h}}\vec{h} = 2b(\vec{W}\cdot\vec{\sigma})\Psi(x)$
- $\eta_1\Psi=\sum_{i,j=1}^MW_iW_j\sum_{\{\vec{h}\}}e^{xb+\vec{c}'\vec{h}+x\vec{W}\vec{h}}h_i h_j=\Psi(x)\sum_{i,j}^M W_iW_j(\delta_{ij}\sigma_i+\sigma_i\sigma_j(1-\delta_{ij}))$

> Careful! defined here different from notes, here each term is $\eta_i\Psi$


In [76]:
function η_1(x)
    return visible_bias^2
end

function η_2(x)
    W_dot_σ=0
    for i in 1:N_hidden
# Error here?
        W_dot_σ = W[i].+σ_i(x,i).+W_dot_σ
    end
    return 2 .*visible_bias.*W_dot_σ
end

function η_3(x)
    total_sum = 0
    for i in 1:N_hidden
        for j in 1:N_hidden
            if i == j
                total_sum = W[i].^2 .* σ_i(x,i) .+ total_sum
            else
                total_sum = W[i]*W[j] .* σ_i(x,i).* σ_i(x,j) .+ total_sum
            end
        end
    end
    return total_sum
end

function kinetic_terms(x)
    return η_1(x).+η_2(x).+η_3(x)
end

kinetic_terms (generic function with 1 method)

Then, the kinetic term $\langle\Psi|\hat{T}|\Psi\rangle = -\frac{1}{2}\Psi(x)\frac{\partial^2}{\partial x^2}\Psi(x)$ can be expressed as:

$$\langle\Psi|\hat{T}|\Psi\rangle=-\frac{1}{2}\Psi(x)^2(\eta_1+\eta_2+\eta_3) $$

## Potential

Included in energy

## Energy

In [98]:
function energy_integrand(x)
    return (Ψ(x)).^2 .* (kinetic_terms(x)+0.5.* x.^2)
end

function energy(a,b,n)
    numerator = simps(energy_integrand, a,b,n)
    denominator = simps(x->(Ψ(x)).^2,a,b,n)
    return numerator/denominator
end

W = rand(N_hidden)

energy(-2,2,10000)

19.850463007111333